In [1]:
from omegaconf import OmegaConf as om
from omegaconf import DictConfig
from typing import cast
import sys 
import os
sys.path.append(os.path.abspath(".."))

from main import build_model
import src.create_bert as bert_module
import src.create_model as model_module
from transformers import AutoTokenizer, Trainer, TrainingArguments



yaml_path = "../yamls/pretrain/micro_dna_monarch-mixer-pretrain-786dim-80m-parameters.yaml"

tokenizer = AutoTokenizer.from_pretrained("gagneurlab/SpeciesLM", revision="downstream_species_lm")


with open(yaml_path) as f:
    cfg = om.load(f)
cfg = cast(DictConfig, cfg)
print(cfg.max_duration)
model = model_module.create_model(cfg.model.get("model_config"))
#model = bert_module.create_bert_mlm(model_config = cfg.model.get("model_config", None))

Could not import FlashFFTConv!


/opt/modules/i12g/anaconda/envs/m2-mixer/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


200000ba
Using Monarch Mixer for Sequence Mixing: True
-- Bidirectional: True
-- Using Long Conv Residual: True
-- Hyena w: 10
-- Hyena w mod: 1
-- Hyena filter order: 128
-- Hyena filter dropout: 0.2
-- Hyena filter wd: 0.1
-- Hyena filter emb dim: 5
-- Hyena filter lr: 0.001
-- Hyena filter lr pos emb: 1e-05


In [2]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(5504, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): MonarchMixerSequenceMixing(
            (filter_fn): HyenaFilter(
              (dropout): Dropout(p=0.2, inplace=False)
              (pos_emb): PositionalEmbedding()
              (implicit_filter): Sequential(
                (0): Linear(in_features=5, out_features=128, bias=True)
                (1): Sin()
                (2): Linear(in_features=128, out_features=128, bias=True)
                (3): Sin()
                (4): Linear(in_features=128, out_features=128, bias=True)
                (5): Sin()
                (6)

In [7]:
for i in range(4103):
    tok = tokenizer.convert_ids_to_tokens(i)
    if tok == '':
        break
    print(f'i: {i}, token: {tok}')

i: 0, token: [PAD]
i: 1, token: [UNK]
i: 2, token: [CLS]
i: 3, token: [SEP]
i: 4, token: [MASK]
i: 5, token: AAAAAA
i: 6, token: AAAAAT
i: 7, token: AAAAAC
i: 8, token: AAAAAG
i: 9, token: AAAATA
i: 10, token: AAAATT
i: 11, token: AAAATC
i: 12, token: AAAATG
i: 13, token: AAAACA
i: 14, token: AAAACT
i: 15, token: AAAACC
i: 16, token: AAAACG
i: 17, token: AAAAGA
i: 18, token: AAAAGT
i: 19, token: AAAAGC
i: 20, token: AAAAGG
i: 21, token: AAATAA
i: 22, token: AAATAT
i: 23, token: AAATAC
i: 24, token: AAATAG
i: 25, token: AAATTA
i: 26, token: AAATTT
i: 27, token: AAATTC
i: 28, token: AAATTG
i: 29, token: AAATCA
i: 30, token: AAATCT
i: 31, token: AAATCC
i: 32, token: AAATCG
i: 33, token: AAATGA
i: 34, token: AAATGT
i: 35, token: AAATGC
i: 36, token: AAATGG
i: 37, token: AAACAA
i: 38, token: AAACAT
i: 39, token: AAACAC
i: 40, token: AAACAG
i: 41, token: AAACTA
i: 42, token: AAACTT
i: 43, token: AAACTC
i: 44, token: AAACTG
i: 45, token: AAACCA
i: 46, token: AAACCT
i: 47, token: AAACCC
i: 48,

In [3]:
from datasets import load_from_disk
from collate import DataCollatorForLanguageModelingSpan

#Load model directly
%env WANDB_PROJECT=singlesamplednam2



dataset = load_from_disk("../micro")
dataset = dataset.remove_columns(["species_name", "__index_level_0__"])

data_collator = DataCollatorForLanguageModelingSpan(tokenizer, mlm=True, mlm_probability = 0.02, span_length = 6)

training_args = TrainingArguments(
    output_dir="./results/correct_model_1sample",
    
    max_steps=100000,
    
    seed=17,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    
    logging_strategy="steps",
    logging_steps=1,
    
    remove_unused_columns=False,
    evaluation_strategy="no",
    
    #dataloader_num_workers=0,
    run_name="repo_model_1sample",
    report_to="wandb"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator
)


#trainer.train()
tokenizer.vocab_size

env: WANDB_PROJECT=singlesamplednam2


4101

In [2]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(4096, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): MonarchMixerSequenceMixing(
            (filter_fn): HyenaFilter(
              (dropout): Dropout(p=0.2, inplace=False)
              (pos_emb): PositionalEmbedding()
              (implicit_filter): Sequential(
                (0): Linear(in_features=5, out_features=128, bias=True)
                (1): Sin()
                (2): Linear(in_features=128, out_features=128, bias=True)
                (3): Sin()
                (4): Linear(in_features=128, out_features=128, bias=True)
                (5): Sin()
                (6): Linear

In [31]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(dataset["train"], batch_size=1, shuffle=True, collate_fn=data_collator)
for step, inputs in enumerate(train_dataloader):
    inputs = {
        "args": inputs
    }
    model(**inputs)

TypeError: HuggingFaceModel.forward() got an unexpected keyword argument 'args'